# Semianr 3 - Applied Quantitative Logistics

## Implementing Genetic Algorithm for *Continuous Problems*

## Continuous GA

**Problem:** Sphere
$$
\min{z} = f_{sph}(x) = \begin{equation*}
 \sum_{i=1}^n {x_i}^2 \end{equation*}
$$

$$
x_{min} \le x_i \le x_{max}
$$

Optimal Solutions:

$$
\forall i \;
\left\{
    \begin{array}\\
        x_i^* = 0 \\
        z^* = 0 \\
    \end{array}
\right.
$$

In [ ]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt

### Cost Function

In [ ]:
def sphere(x):
    
    global NFE
    
    if pd.isna(NFE):
        NFE = 0
    
    NFE += 1
    
    z = [item**2 for item in x]
    
    return sum(z)

In [ ]:
def pop_sort(p, c):
    
    li = [[c[i], i] for i in range(len(c))]
    li.sort()
    
    sorted_index = [x[1] for x in li]
    
    sorted_pop = [p[i] for i in sorted_index]
    sorted_cost = [c[i] for i in sorted_index]

    return sorted_pop, sorted_cost

### CrossOver

In [ ]:
def CrossOver(x1, x2, gamma, varMin, varMax):
    
    alpha = list(np.random.uniform(-gamma, 1+gamma, size=len(x1)))
    
    y1 = list(np.multiply(alpha, x1) + (1 - np.array(alpha))*np.array(x2))
    y2 = list(np.multiply(alpha, x2) + (1 - np.array(alpha))*np.array(x1))
    
    y1 = [max(i, varMin) for i in y1]
    y1 = [min(i, varMax) for i in y1]
    
    y2 = [max(i, varMin) for i in y2]
    y2 = [min(i, varMax) for i in y2]
    
    return y1, y2

### Mutation

In [ ]:
def mutation(x, varMin, varMax):
    index = int(np.random.randint(0, len(x), size=1))
    
    sigma = 0.1 * (varMax-varMin)
    
    y = x.copy()
    
    y[index] = x[index] + sigma*np.random.randn()
    
    y = [max(i, varMin) for i in y]
    y = [min(i, varMax) for i in y]    
    return y

## How to more improve the Genetic Algorithm 

Parent selection can be an option.

### Roulette Wheel Selection

In [ ]:
def rouletteWheelSelection(p):
    r = random.random()
    
    c = np.cumsum(p)
    
    indexes = [
    index for index in range(len(c))
    if c[index] > r
    ]
    
    return indexes[0]

### Tournoment Selection

In [ ]:
def tournomentSelection(pop, costs, m):
    
    S = random.sample(range(0, len(pop)-1), 3)
    
    spop = [[pop[k]] for k in S]
    scosts = [[costs[k]] for k in S]
    
    min_ = min(scosts)
    i = scosts.index(min_)
    
    return i

#### Important fact

In calculating the probability we said:
$$
p_i \; \alpha \; \exp^{-\beta c_i}
$$

Let's say we want to minimize the cost of a project, first it was based on $, then RUB, and then maybe with another currency which has the lowest value. In this case the scale for the function will change.

When we change the data (cost scale changes), then we need to change the **betha** too. In this senario we manipulate the formula to normalize it:

$$
p_i \; \alpha \; \exp^{-\beta \frac{c_i}{c_{max}}}
$$

$c_{max}$ is the worst cost ever found



In this case with one setting we solve different types of problems. Now **betha** is independant from the cost function.

In [ ]:
### Problem Parameters Definition ###
nVar = 5       # Number of decision variables

varMin = -10   # Lower Bound of Variables
varMax = 10    # Upper Bound of Variables

global NFE
NFE = 0

### GA Parameters ###
maxIt = 200     # Maximum numner of iterations
nPop = 100       # Population size 

pc = 0.8                   # Crossover percentage
nc = 2*round(pc*nPop/2)    # Number of offsprings (parents)

pm = 0.3                   # Mutation percentage
nm = round(pm*nPop)        # Number of mutants

gamma = 0.05

beta = 8
tournomentSize = 3

# Make option for roulette wheel or tournoment
UseRouletteWheelSelection = UseTournomentSelection = UseRandomSelection = None
Ans  =input("Choose selection method: (Roulette Wheel / Tournoment / Random)?  ")
if Ans == "Roulette Wheel":
    UseRouletteWheelSelection = True
elif Ans == "Tournoment":
    UseTournomentSelection = True
elif Ans == "Random":
    UseRandomSelection = True

# for roulette wheele selection
if UseRouletteWheelSelection:
    beta = 8       # Selection pressure
    
# for tournoment selection
if UseTournomentSelection:
    tournomentSize = 3         # Tournoment size

### Initialization ###
pop, costs = [], []

for i in range(0, nPop):
    pop.append(list(np.random.uniform(varMin, varMax, size=nVar)))
    costs.append(sphere(pop[i]))

# Sort the population and costs
pop, costs = pop_sort(pop, costs)

#  Store the best solution
bestSolution = [pop[0]]

# Array to hold best cost values
bestCosts = [costs[0]]

# Store the worst cost
worstCost = costs[-1]

# Array to hold number of function evaluation
nfe = [NFE]

### Main Loop ###
for it in range(1, maxIt):
      
    # Calculate selection probabilities
#     P = [math.exp(-beta*item) for item in costs]
#     P = list(np.array(P) / sum(P))

    P = [math.exp(-beta*(item/worstCost)) for item in costs]
    P = list(np.array(P) / sum(P))

    # Crossover
    popc, popc_cost = [], []
    for k in range(1, int(nc/2)):
        
        
        # Select parents indices
#         rand1 = int(np.random.randint(nPop, size=1))
#         rand2 = int(np.random.randint(nPop, size=1))

#         rand1 = rouletteWheelSelection(P)
#         rand2 = rouletteWheelSelection(P)

#         rand1 = tournomentSelection(pop, costs, tournomentSize)
#         rand2 = tournomentSelection(pop, costs, tournomentSize)

        if UseRouletteWheelSelection:
            rand1 = rouletteWheelSelection(P)
            rand2 = rouletteWheelSelection(P)
        if UseTournomentSelection:
            rand1 = tournomentSelection(pop, costs, tournomentSize)
            rand2 = tournomentSelection(pop, costs, tournomentSize)
        if UseRandomSelection:
            rand1 = int(np.random.randint(nPop, size=1))
            rand2 = int(np.random.randint(nPop, size=1))
            
   
        # Select parents
        p1 = pop[rand1]
        p2 = pop[rand2]
        
        # Apply crossover
        y1, y2 = CrossOver(p1, p2, gamma, varMin, varMax)
        
        popc.append(y1)
        popc.append(y2)
        
        # Evaluate offsprings
        popc_cost.append(sphere(y1))
        popc_cost.append(sphere(y2))
    
    popm, popm_cost = [], []
    
    # Mutation
    for k in range(1, nm):
        
        # Select parent
        rand = int(np.random.randint(nPop, size=1))
        p = pop[rand]
        
        # Apply mutation
        popm.append(mutation(p, varMax, varMin))

        # Evaluate mutate
        popm_cost.append(sphere(popm[-1]))
        
    # Create merged population
    pop = pop + popc + popm
    costs = costs + popc_cost + popm_cost
    
    # Sort the population and costs
    pop, costs = pop_sort(pop, costs)
    
    # Truncation
    pop = pop[:nPop]
    costs = costs[:nPop]
    
    # Store best solution ever found
    bestSolution.append(pop[0])
    
    # Store best cost ever found
    bestCosts.append(costs[0])
     
    
    # Update the worst cost
    worstCost = max(worstCost, costs[-1])


    # Store nfe
    nfe.append(NFE)
    
#     if bestCosts[-2] == 0:
#         break
    
#     print(f'Iteration {it} : Best Cost = {bestCosts[it]}') 
    print(f'Iteration {it} : NFE = {nfe[it]},  Best Cost = {bestCosts[it]}')

### Plot the results

In [ ]:
plt.plot(nfe, bestCosts, linewidth = 3)
plt.xlabel('NFE')
plt.ylabel('Best Cost')

### y-axis in logarithm

In [ ]:
plt.plot(nfe, np.log(bestCosts), linewidth = 3)
plt.xlabel('NFE')
plt.ylabel('Best Cost')